<a href="https://colab.research.google.com/github/vishaljoshi24/DnD-AutoPrompt/blob/main/classifying_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U dspy

SET-UP

In [ ]:
import dspy

lm = dspy.LM("ollama_chat/llama3.2:1b", api_base = "http://localhost:11434", api_key="b229890ea0664f6193770b4b470f3e74.nJRqYf9TBF9MX24r4fUQQqqB")
dspy.configure(lm=lm)

DATA

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("garrykuwanto/crd3_training_pairs", download_mode="force_redownload")

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
query = []
passage = []
for i in range(len(df.train[0:])):
  query.append(df.train[i]['query'])
  passage.append(df.train[i]['passage'])

df1 = pd.DataFrame({'context': query, 'current turn': passage})

In [ ]:
df1

In [ ]:
df2 = df1[500:510]

df2

In [ ]:
df2.insert(2, 'quality', ['goal-oriented', 'DM turn', 'goal-oriented', 'DM turn', 'collaborative action', 'open ended', 'contextually relevant', 'collaborative', 'goal-oriented', 'open ended'  ])

In [ ]:
examples = []

for context, turn, quality in df2.values:
  examples.append(dspy.Example(context=context, turn=turn, quality=quality).with_inputs("context", "current turn", "quality"))

examples

SIGNATURES

In [ ]:
from typing import Literal


from typing import Literal

class TurnClassifier(dspy.Signature):
    data: dict[str, str] = dspy.InputField(desc = """
  {'context': 'The three previous game turns which describe a player's action or their dialogue.',
   'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
   }""")
    behaviour_class: Literal['collaborative',
                             'contextually-relevant',
                             'goal-oriented',
                             'open-ended'] = dspy.OutputField(desc="The feature of a player's turn within a Dungeons & Dragons game: collaborative, open-ended, goal-oriented, contextually relevant")

classify = dspy.ChainOfThought(BehaviourClassifier)

# class PlayerInstruction(dspy.Signature):
#   "Generate a prompt which instructs an agent how to behave as a D&D player based on the labelled quality for a specific gameplay turn"

#   examples: dict[str, str] = dspy.InputField(desc = """
#   {'context': 'The three previous game turns which describe a player's action or their dialogue.',
#    'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
#    'behaviour': 'The behavious exhibited by a player in their turn within a Dungeons & Dragons game: open ended, goal oriented, contextually relevant and collaborative. DM turns should be ignored.'
#   }
#   """)
#   instruction: str = dspy.OutputField(desc = "instruction on how to behave within a D&D game which describes the quality label of a particular turn.")

  # context: dict[str, str] = dspy.InputField(desc = "The three previous game turns which describe a player's action or their dialogue")
  # turn: dict[str,str] = dspy.InputField(desc = "The current turn taken by a player, which can include a description of an action or a piece of dialogue.")
  # quality: dict[str,str] = dspy.InputField(desc="The quality of a player's actions within a Dungeons & Dragons game: open-ended, goal-oriented, contextually relevant, collaborative.")




MODULES

In [ ]:
# class PromptGenerator(dspy.Module):
#   def __init__(self):
#     # self.classifier = dspy.ChainOfThought(Quality, caching = False)
#     self.generator = dspy.ChainOfThought(PlayerInstruction, caching = False)

#   def forward(self, examples, **kwargs):
#     # quality_class = self.classifier(action=action).quality
#     prompt = self.generator(examples=examples).instruction
#     return prompt

In [ ]:
# generate_prompts = PromptGenerator()
# prompt_list = []
# for i in range(len(examples)):
#   prompt_list.append(generate_prompts(i))
# prompt_list

EVALUATION QUESTIONS



  clear = "Is the assessed prompt specific, unambiguous and easy to interpret?"

  relevant = "Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?"

  correct = "Is the assessed prompt correct with respect to the player quality that it is describing?"

  complete = "Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?"



EVALUATION SIGNATURES

In [ ]:
# class Assess(dspy.Signature):
#   """Assess the quality of a prompt along a specified dimension."""

#   assessed_text: list[str] = dspy.InputField()
#   assessment_question: str = dspy.InputField()
#   assessment_answer: bool = dspy.OutputField()

In [ ]:
# clarity_evaluation = dspy.ChainOfThought(Assess, caching=False)
# relevance_evaluation = dspy.ChainOfThought(Assess, caching=False)
# correctness_evaluation = dspy.ChainOfThought(Assess, caching=False)
# completeness_evaluation = dspy.ChainOfThought(Assess, caching=False)

In [ ]:
# clarity_list = []
# for i in prompt_list:
#   prompt_clarity = clarity_evaluation(assessed_text=i, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?")
#   clarity_list.append(prompt_clarity)

# clarity_list

# # clarity_evaluation(assessed_text=clarity_list, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?")

In [ ]:
# relevance_list = []
# for i in prompt_list:
#   prompt_relevance = relevance_evaluation(assessed_text=i, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?")
#   relevance_list.append(prompt_relevance)

# relevance_list

# # relevance_evaluation(assessed_text=example_1, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?")

In [ ]:
# correctness_list = []
# for i in prompt_list:
#   prompt_correctness = correctness_evaluation(assessed_text=i, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?")
#   correctness_list.append(prompt_correctness)

# correctness_list

# # correctness_evaluation(assessed_text=example_1, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?")

In [ ]:
# completeness_list = []
# for i in prompt_list:
#   prompt_completeness = completeness_evaluation(assessed_text=i, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?")
#   completeness_list.append(prompt_completeness)

# completeness_list

# # completeness_evaluation(assessed_text=example_1, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?")

In [ ]:
# class ScorePrompt(dspy.Module):
#   def __init__(self):
#     self.assessment = dspy.ChainOfThought(Assess, caching=False)

#   def forward(self, prompt_list, **kwargs):
#     score_list = []
#     for i in prompt_list:
#       assess_clarity = self.assessment(assessed_text=i, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?").assessment_answer
#       assess_relevance = self.assessment(assessed_text=i, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?").assessment_answer
#       assess_correctness = self.assessment(assessed_text=i, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?").assessment_answer
#       assess_completeness = self.assessment(assessed_text=i, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?").assessment_answer

#       if assess_clarity == True:
#         assess_clarity = 1
#       else:
#         assess_clarity = 0
#       if assess_relevance == True:
#         assess_relevance = 1
#       else:
#         assess_relevance = 0
#       if assess_correctness == True:
#         assess_correctness = 1
#       else:
#         assess_correctness = 0
#       if assess_completeness == True:
#         assess_completeness = 1
#       else:
#         assess_completeness = 0

#       score = (assess_clarity + assess_relevance + assess_correctness + assess_completeness)/4
#       score_list.append(score)

#     return score_list

In [ ]:
# prompt_score_list = ScorePrompt()
# prompt_score_list(prompt_list=prompt_list)